<a href="https://colab.research.google.com/github/aliayehya/AltinbasMasterThesis/blob/main/age_prediction_web_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!gdown --id 1qPv_3q3Gmuo1mZwE-M5dirdj13lGVbXi
!unzip B4-Regression.zip

#download face detection model
!gdown --id 1uXfqiInKdhrluOhtKwneXav5YZ81MCG7




Downloading...
From: https://drive.google.com/uc?id=1qPv_3q3Gmuo1mZwE-M5dirdj13lGVbXi
To: /content/B4-Regression.zip
100% 66.7M/66.7M [00:00<00:00, 128MB/s] 
Archive:  B4-Regression.zip
   creating: content/My_model/
  inflating: content/My_model/saved_model.pb  
  inflating: content/My_model/keras_metadata.pb  
   creating: content/My_model/assets/
   creating: content/My_model/variables/
  inflating: content/My_model/variables/variables.data-00000-of-00001  
  inflating: content/My_model/variables/variables.index  
Downloading...
From: https://drive.google.com/uc?id=1uXfqiInKdhrluOhtKwneXav5YZ81MCG7
To: /content/mmod_human_face_detector.dat
100% 730k/730k [00:00<00:00, 48.4MB/s]


In [2]:

#importing libraries
import cv2
from albumentations import( Compose,Resize, ToFloat)
import numpy as np
from keras.models import load_model
from keras.metrics import mean_absolute_error
from google.colab import files
import matplotlib.pyplot as plt
import dlib



#augmentations
AUGMENTATIONS_TEST = Compose([
    Resize(200, 200, always_apply=False, p=1),
    ToFloat(max_value=255)])

In [3]:
!pip install flask-ngrok
!pip install flask==0.12.2  # Newer versions of flask don't work in Colab
                            # See https://github.com/plotly/dash/issues/257
! pip install pyngrok
! ngrok authtoken 229evKfeI1i6q0fCWLRL4BdpbcB_36Q4i5wKZPcGS8LnC9XW2


     |████████████████████████████████| 83 kB 437 kB/s 
  Attempting uninstall: flask
    Found existing installation: Flask 1.1.4
    Uninstalling Flask-1.1.4:
      Successfully uninstalled Flask-1.1.4
     |████████████████████████████████| 745 kB 5.0 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=9ae5dee67a47d09547f0042125525b0847643d9b9ca4b3609fc720befd2d4030
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [4]:
checkpoint_path = "/content/content/My_model"
def mae_years(in_gt, in_pred):
    return mean_absolute_error(in_gt, in_pred)
model = load_model(checkpoint_path, custom_objects={'mae_years':mae_years})
cnn_face_detector = dlib.get_frontal_face_detector()
       
print('Model loaded')

Model loaded


In [14]:
# flask_ngrok_example.py
# Flask utils
from flask import Flask, redirect, url_for, request, render_template
from werkzeug.utils import secure_filename
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app is run

def model_predict(img_path, model):
    Image = cv2.imread(img_path, cv2.COLOR_BGR2RGB)
    # apply face detection (cnn)
    faces_cnn = cnn_face_detector(Image, 1)
    print(faces_cnn)
    if len(faces_cnn) >= 1:
      print('face deteted')
      face = faces_cnn[0]
      x1, y1= face.left(), face.top()
      x2, y2 = face.right(), face.bottom()
      Image = Image[y1:y2,x1:x2,:] 
    img_augmented = AUGMENTATIONS_TEST(image=Image)
    img = np.expand_dims(img_augmented['image'], axis=0)
    img.shape
    preds = model.predict(img, verbose = True)
    return preds

@app.route('/', methods=['GET']) 
def hello_word(): 
    return render_template('index.html') 
@app.route('/', methods=['POST']) 
def predict(): 
    imagefile= request.files['imagefile'] 
    image_path = "/content/templates/uploads/" + imagefile.filename 
    imagefile.save(image_path)  
    Image = cv2.imread(image_path, cv2.COLOR_BGR2RGB)
    # apply face detection (cnn)
    faces_cnn = cnn_face_detector(Image, 1)
    print(faces_cnn)
    if len(faces_cnn) >= 1:
      print('face deteted')
      face = faces_cnn[0]
      x1, y1= face.left(), face.top()
      x2, y2 = face.right(), face.bottom()
      Image = Image[y1:y2,x1:x2,:] 
    img_augmented = AUGMENTATIONS_TEST(image=Image)
    img = np.expand_dims(img_augmented['image'], axis=0)
    img.shape
    preds = model.predict(img, verbose = True)
    age = preds[0][0]
    return render_template('index.html',prediction=round(age),user_image =  image_path)
    
if __name__ == '__main__':
    app.run()  # If address is in use, may need to terminate other sessions:
               # Runtime > Manage Sessions > Terminate Other Sessions

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://0bef-35-185-37-166.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [22/Jan/2022 20:46:53] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Jan/2022 20:46:53] "GET /favicon.ico HTTP/1.1" 404 -


rectangles[[(58, 428) (724, 1094)]]
face deteted
1/1 [==============================] - 0s 170ms/step


127.0.0.1 - - [22/Jan/2022 20:47:06] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [22/Jan/2022 20:47:06] "GET /content/templates/uploads/photo_2021-12-09_19-53-59.jpg HTTP/1.1" 404 -
